In [ ]:
#pip install datasets langchain_community rank_bm25 langgraph langchain_huggingface duckduckgo-search

**Step 1: Load and Prepare the Dataset**

We will use the Hugging Face datasets library to load the dataset and convert it into a list of Document objects from the langchain.docstore.document module.

In [1]:
import datasets
from langchain.docstore.document import Document

# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]


C:\Users\gyane\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\gyane\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gyane\.cache\huggingface\hub\datasets--agents-course--unit3-invitees. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python

In the code above, we:

Load the dataset

Convert each guest entry into a Document object with formatted content

Store the Document objects in a list

**Step 2: Create the Retriever Tool**

We will use the BM25Retriever from the langchain_community.retrievers module to create a retriever tool.

The BM25Retriever is a great starting point for retrieval, but for more advanced semantic search, you might consider using embedding-based retrievers like those from sentence-transformers.

In [2]:
from langchain_community.retrievers import BM25Retriever
from langchain.tools import Tool

bm25_retriever = BM25Retriever.from_documents(docs)

def extract_text(query: str) -> str:
    """Retrieves detailed information about gala guests based on their name or relation."""
    results = bm25_retriever.invoke(query)
    if results:
        return "\n\n".join([doc.text for doc in results[:3]])
    else:
        return "No matching guest information found."

guest_info_tool = Tool(
    name="guest_info_retriever",
    func=extract_text,
    description="Retrieves detailed information about gala guests based on their name or relation."
)

Let’s understand this tool step-by-step.



*   The name and description help the agent understand when and how to use this tool.
*   The inputs define what parameters the tool expects (in this case, a search query)
*   We’re using a BM25Retriever, which is a powerful text retrieval algorithm that doesn’t require embeddings
*   The forward method processes the query and returns the most relevant guest information

**Step 3: Integrate the Tool with Alfred**

In [5]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

# Generate the chat interface, including the tools
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
    huggingfacehub_api_token="token-goes-here",
)

chat = ChatHuggingFace(llm=llm, verbose=True)
tools = [guest_info_tool]
chat_with_tools = chat.bind_tools(tools)

# Generate the AgentState and Agent graph
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }

## The graph
builder = StateGraph(AgentState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message requires a tool, route to tools
    # Otherwise, provide a direct response
    tools_condition,
)
builder.add_edge("tools", "assistant")
alfred = builder.compile()

messages = [HumanMessage(content="Tell me about our guest named 'Lady Ada Lovelace'.")]
response = alfred.invoke({"messages": messages})

print("🎩 Alfred's Response:")
print(messages['messages'][-1].content)

C:\Users\gyane\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gyane\.cache\huggingface\hub\models--Qwen--Qwen2.5-Coder-32B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

**Give Your Agent Access to the Web**

Remember that we want Alfred to establish his presence as a true renaissance host, with a deep knowledge of the world.

To do so, we need to make sure that Alfred has access to the latest news and information about the world.

Let’s start by creating a web search tool for Alfred!

In [6]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke("Who's the current President of France?")
print(results)

Emmanuel Macron is a French banker and politician who was elected president of France in 2017. Macron was the first person in the history of the Fifth Republic to win the presidency without the backing of either the Socialists or the Gaullists, and he was France's youngest head of state since Napoleon. Find out who the current president of France is, his political career, his actions, and his impact on the country. Stay informed about French news and presidential decisions. The current President of France is Emmanuel Macron, who has held office since being elected in the 2017 French Presidential Election. The Prime Minister of France is the leader of government and holds the power to manage the numerous public agencies based around the nation. PARIS (AP) — French President Emmanuel Macron vowed Thursday to stay in office until the end of his term, due in 2027, and announced that he will name a new prime minister within days following ... France has a semi-presidential system of governm

**Creating a Custom Tool for Weather Information to Schedule the Fireworks**

The perfect gala would have fireworks over a clear sky, se need to make sure the fireworks are not cancelled due to bad weather.

Let’s create a custom tool that can be used to call an external weather API and get the weather information for a given location.

In [7]:
from langchain.tools import Tool
import random

def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location."""
    # Dummy weather data
    weather_conditions = [
        {"condition": "Rainy", "temp_c": 15},
        {"condition": "Clear", "temp_c": 25},
        {"condition": "Windy", "temp_c": 20}
    ]
    # Randomly select a weather condition
    data = random.choice(weather_conditions)
    return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

# Initialize the tool
weather_info_tool = Tool(
    name="get_weather_info",
    func=get_weather_info,
    description="Fetches dummy weather information for a given location."
)

**Creating a Hub Stats Tool for Influential AI Builders**


In attendance at the gala are the who’s who of AI builders. Alfred wants to impress them by discussing their most popular models, datasets, and spaces. We’ll create a tool to fetch model statistics from the Hugging Face Hub based on a username.

In [8]:
from langchain.tools import Tool
from huggingface_hub import list_models

def get_hub_stats(author: str) -> str:
    """Fetches the most downloaded model from a specific author on the Hugging Face Hub."""
    try:
        # List models from the specified author, sorted by downloads
        models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

        if models:
            model = models[0]
            return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
        else:
            return f"No models found for author {author}."
    except Exception as e:
        return f"Error fetching models for {author}: {str(e)}"

# Initialize the tool
hub_stats_tool = Tool(
    name="get_hub_stats",
    func=get_hub_stats,
    description="Fetches the most downloaded model from a specific author on the Hugging Face Hub."
)

# Example usage
print(hub_stats_tool("facebook")) # Example: Get the most downloaded model by Facebook

The most downloaded model by facebook is facebook/esmfold_v1 with 16,832,501 downloads.


C:\Users\gyane\AppData\Local\Temp\ipykernel_6160\3056453286.py:26: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(hub_stats_tool("facebook")) # Example: Get the most downloaded model by Facebook


**Integrating Tools with Alfred**

Now that we have all the tools, let’s integrate them into Alfred’s agent:

In [10]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

# Generate the chat interface, including the tools
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
    huggingfacehub_api_token="hf_token goes here",
)

chat = ChatHuggingFace(llm=llm, verbose=True)
tools = [search_tool, weather_info_tool, hub_stats_tool]
chat_with_tools = chat.bind_tools(tools)

# Generate the AgentState and Agent graph
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }

## The graph
builder = StateGraph(AgentState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message requires a tool, route to tools
    # Otherwise, provide a direct response
    tools_condition,
)
builder.add_edge("tools", "assistant")
alfred = builder.compile()

messages = [HumanMessage(content="Who is Facebook and what's their most popular model?")]
response = alfred.invoke({"messages": messages})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [11]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_community.tools import DuckDuckGoSearchRun



In [13]:
# Initialize the web search tool
search_tool = DuckDuckGoSearchRun()

# Load the guest dataset and initialize the guest info tool
guest_info_tool = guest_info_tool

# Generate the chat interface, including the tools
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
    huggingfacehub_api_token="hf_token-goes-here",
)

chat = ChatHuggingFace(llm=llm, verbose=True)
tools = [guest_info_tool, search_tool, weather_info_tool, hub_stats_tool]
chat_with_tools = chat.bind_tools(tools)

# Generate the AgentState and Agent graph
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }

## The graph
builder = StateGraph(AgentState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message requires a tool, route to tools
    # Otherwise, provide a direct response
    tools_condition,
)
builder.add_edge("tools", "assistant")
alfred = builder.compile()

In [14]:
response = alfred.invoke({"messages": "Tell me about 'Lady Ada Lovelace'"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:
Ada Lovelace, born Augusta Ada Byron on December 10, 1815, was the only legitimate child of the famous poet, Lord Byron, and Annabella Milbanke. Her mother had a strong interest in mathematics and logic, which led her to encourage Ada to pursue these subjects. This early exposure to mathematics likely played a significant role in shaping Ada's academic and professional life.

In the 1840s, Ada met Charles Babbage, a mathematician renowned for his work on mechanical computing devices, most famously the Analytical Engine, which was an early conceptualization for a computer. The Analytical Engine could perform complex calculations, store data, and be "programmed" with punched cards, much like early computers did a century later.

Lovelace collaborated with Babbage, translating an article from French about the Analytical Engine into English and appending her own detailed comments, including a set of notes that are now regarded as the first algorithm intended to be proc

In [15]:
response = alfred.invoke({"messages": "What's the weather like in Paris tonight? Will it be suitable for our fireworks display?"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:
Given that the weather in Paris tonight is expected to be windy with a temperature of 20°C, it might not be the most ideal conditions for a fireworks display. The wind could potentially scatter the fireworks, making the show less impressive and harder to enjoy. You might want to consider rescheduling the fireworks display to a night with clearer and calmer conditions for the best viewing experience.


In [16]:
response = alfred.invoke({"messages": "One of our guests is from Qwen. What can you tell me about their most popular model?"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:
It seems there might have been a misunderstanding. According to the information available, Qwen is a large language model created by Alibaba Cloud, and the specific model details you were inquiring about may not have been correctly referenced.

As of my last update, Qwen includes a variety of models optimized for different tasks, such as text generation, translation, and more. If "Qwen/Qwen2.5-7B-Instruct" is a specific variant you're referring to, it seems to be very popular, with over 2 million downloads, as indicated in your initial message.

However, if you need specific information about the most popular Qwen model, I recommend checking the latest releases and download statistics on the official platform or repository where Qwen models are hosted, such as Hugging Face Transformers or Alibaba Cloud's repository.

If you have more details or another specific model in mind, feel free to provide them, and I can try to provide more accurate information!


In [20]:
response = alfred.invoke({"messages":"I need to speak with 'Dr. Nikola Tesla' about recent advancements in wireless energy. Can you help me prepare for this conversation?"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:



In [19]:
# First interaction
response = alfred.invoke({"messages": [HumanMessage(content="Tell me about 'Lady Ada Lovelace'. What's her background and how is she related to me?")]})


print("🎩 Alfred's Response:")
print(response['messages'][-1].content)
print()

# Second interaction (referencing the first)
response = alfred.invoke({"messages": response["messages"] + [HumanMessage(content="What projects is she currently working on?")]})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:
Ada Lovelace, born Augusta Ada Byron on December 10, 1815, and later known as the Countess of Lovelace, is celebrated as one of the first computer programmers. She was the daughter of the famous poet Lord Byron and Annabella Milbanke, a woman who had a strong interest in mathematics and logic, which likely influenced Ada's early life and education.

Ada's mother feared that Ada would inherit her father's tumultuous nature and focused on scientific, mathematical, and musical education to steer her away from poetry. Under her mother's supervision, Ada received rigorous tutoring in mathematics, a subject she excelled in. Her aptitude for numbers caught the attention of Charles Babbage, a leading English mathematician and inventor. Babbage was impressed by Ada’s intellect and invited her to attend his lectures at the University of London.

Babbage was working on the Analytical Engine, a mechanical device designed to perform complex mathematical calculations. Many consi